In [2]:
import re
import string

!pip install -U nltk

import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize # Sentence Tokenizer
from nltk.tokenize import word_tokenize # Word Tokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

Requirement already up-to-date: nltk in c:\users\mhdal\anaconda3\envs\lambda-nlp\lib\site-packages (3.4.4)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mhdal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mhdal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:

train = pd.read_csv('train.csv')

In [16]:
train.isnull().sum()

id                0
author            0
description       0
price            63
ratingValue       0
pert_alcohol     60
category        288
dtype: int64

In [12]:
train.shape

(2874, 7)

In [377]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category,tokens,tokenized_sentences,lemmas,lammatized_sentence,string_category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0,"[marriage, 13, 18, year, old, bourbons., a, ma...",marriage 13 18 year old bourbons. a mature el...,"[ , marriage, 13, 18, year, old, bourbon, a, m...",marriage 13 18 year old bourbon a mature el...,Bourbon/Tennessee
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0,"[legendary, bowmores, mid-60s, bit, equal., al...",legendary bowmores mid-60s bit equal. all sha...,"[ , legendary, bowmore, mid-60, bit, equal, al...",legendary bowmore mid-60 bit equal all shar...,Scotch
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0,"[bottling, celebrates, master, distiller, park...",bottling celebrates master distiller parker b...,"[ , bottling, celebrate, master, distiller, pa...",bottling celebrate master distiller parker ...,Bourbon/Tennessee
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0,"[impresses, whisky, evolves;, incredibly, comp...",impresses whisky evolves; incredibly complex....,"[ , impress, whisky, evolve, incredibly, compl...",impress whisky evolve incredibly complex on...,Scotch
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0,"[caramel-laden, fruit, bouquet,, followed, une...","caramel-laden fruit bouquet, followed unendin...","[ , caramel, laden, fruit, bouquet, follow, un...",caramel laden fruit bouquet follow unending...,Bourbon/Tennessee


In [18]:
train = train.dropna(subset=['category'])
train.isnull().sum()

id               0
author           0
description      0
price           54
ratingValue      0
pert_alcohol    56
category         0
dtype: int64

In [301]:
train = train.fillna(method='ffill')

In [379]:
train = train.reset_index().drop('index', axis=1)

In [397]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category,tokens,tokenized_sentences,lemmas,lammatized_sentence,string_category
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0,"[marriage, 13, 18, year, old, bourbons., a, ma...",marriage 13 18 year old bourbons. a mature el...,"[ , marriage, 13, 18, year, old, bourbon, a, m...",marriage 13 18 year old bourbon a mature el...,Bourbon/Tennessee
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0,"[legendary, bowmores, mid-60s, bit, equal., al...",legendary bowmores mid-60s bit equal. all sha...,"[ , legendary, bowmore, mid-60, bit, equal, al...",legendary bowmore mid-60 bit equal all shar...,Scotch
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0,"[bottling, celebrates, master, distiller, park...",bottling celebrates master distiller parker b...,"[ , bottling, celebrate, master, distiller, pa...",bottling celebrate master distiller parker ...,Bourbon/Tennessee
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0,"[impresses, whisky, evolves;, incredibly, comp...",impresses whisky evolves; incredibly complex....,"[ , impress, whisky, evolve, incredibly, compl...",impress whisky evolve incredibly complex on...,Scotch
4,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0,"[caramel-laden, fruit, bouquet,, followed, une...","caramel-laden fruit bouquet, followed unendin...","[ , caramel, laden, fruit, bouquet, follow, un...",caramel laden fruit bouquet follow unending...,Bourbon/Tennessee


In [167]:
cagtegory_mapper = {1:    'Scotch',
2:    'Bourbon/Tennessee',
3:    'Craft Whiskey',
4:    'Canadian',}

In [169]:
train['string_category'] = train.replace({"category": cagtegory_mapper})['category']

In [19]:
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

nlp = spacy.load("en_core_web_lg")

In [490]:
def lemmas_sentence(data, series):
    tokenizer = Tokenizer(nlp.vocab)
    STOP_WORDS = nlp.Defaults.stop_words.union([' ','I', 'i', 'and', 'it', "it's", 'it.', 'the', 'this','of', 'is', 'my', 'but'])
    tokens = []

    """ Make them tokens """
    for doc in tokenizer.pipe(data[series], batch_size=500):
        doc_tokens = []
        for token in doc[1:]:
            if token.text not in STOP_WORDS:
                doc_tokens.append(token.text.lower())
        tokens.append(doc_tokens)
        
            
    data['tokens'] = tokens
    list_string = []
    for i in data['tokens']:
        stringe = ""
        for string in i:
            stringe = stringe + ' ' + string
        list_string.append(stringe)
    data['tokenized_sentences'] = list_string
    
    def get_lemmas(text):

        lemmas = []

        doc = nlp(text)

        # Something goes here :P
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
                lemmas.append(token.lemma_)

        return lemmas

    data['lemmas'] = data['tokenized_sentences'].apply(get_lemmas)
    
    list_string = []
    for i in data['lemmas']:
        stringe = ""
        for string in i:
            stringe = stringe + ' ' + string
        list_string.append(stringe)
        
    data['lammatized_sentence'] = list_string
    
    return data

In [491]:
testing = lemmas_sentence(test, 'description')
train = lemmas_sentence(train, 'description')

In [492]:
def lemmas_sentence(data, series):
    
    def get_lemmas(text):

        lemmas = []

        doc = nlp(text)

        # Something goes here :P
        for token in doc: 
            if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
                lemmas.append(token.lemma_)

        return lemmas

    data['lemmas'] = data[series].apply(get_lemmas)
    
    list_string = []
    for i in data['lemmas']:
        stringe = ""
        for string in i:
            stringe = stringe + ' ' + string
        list_string.append(stringe)
        
    data['lammatized_author'] = list_string
    
    return data

In [493]:
testing = lemmas_sentence(testing, 'author')
train = lemmas_sentence(train, 'author')

In [462]:
testing.head(), train.head()

(     id                author  \
 0   955          Fred Minnick   
 1  3532            Lew Bryson   
 2  1390  Davin de Kergommeaux   
 3  1024           Gavin Smith   
 4  1902           Gavin Smith   
 
                                          description  price  ratingValue  \
 0  Think carnival aromas—the good ones, anyway—me...   36.0           90   
 1  A blend of three bourbons, between 6 and 12 ye...   90.0           82   
 2  The nose is focused on cereal, hints of fresh ...   48.0           89   
 3  Swiss-based Chapter 7 released this 19 year ol...  180.0           90   
 4  Valkyrie replaces the current Dark Origins exp...   71.0           87   
 
    pert_alcohol             tokens tokenized_sentences  \
 0          50.0          [minnick]             minnick   
 1          49.3           [bryson]              bryson   
 2          45.0  [de, kergommeaux]      de kergommeaux   
 3          55.8            [smith]               smith   
 4          45.9            [smith]

In [20]:
tokenizer = Tokenizer(nlp.vocab)

In [21]:
STOP_WORDS = nlp.Defaults.stop_words.union([' ','I', 'i', 'and', 'it', "it's", 'it.', 'the', 'this','of', 'is', 'my', 'but'])

In [22]:
# Tokenizer Pipe

tokens = []

""" Make them tokens """
for doc in tokenizer.pipe(train['description'], batch_size=500):
    doc_tokens = []
    for token in doc[1:]:
        if token.text not in STOP_WORDS:
            doc_tokens.append(token.text.lower())
    tokens.append(doc_tokens)
        
            
train['tokens'] = tokens

In [23]:
list_string = []
for i in train['tokens']:
    stringe = ""
    for string in i:
        stringe = stringe + ' ' + string
    list_string.append(stringe)

In [24]:
train['tokenized_sentences'] = list_string

In [25]:
def get_lemmas(text):

    lemmas = []
    
    doc = nlp(text)
    
    # Something goes here :P
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False)) and (token.pos_!= 'PRON'):
            lemmas.append(token.lemma_)
    
    return lemmas

In [26]:
train['lemmas'] = train['tokenized_sentences'].apply(get_lemmas)

In [27]:
list_string = []
for i in train['lemmas']:
    stringe = ""
    for string in i:
        stringe = stringe + ' ' + string
    list_string.append(stringe)

In [28]:
train['lammatized_sentence'] = list_string

In [37]:
train.head()

,id,author,description,price,ratingValue,pert_alcohol,category,tokens,tokenized_sentences,lemmas,lammatized_sentence
0,1,John Hansell,A marriage of 13 and 18 year old bourbons. A m...,85.0,97,51.50,2.0,"[marriage, 13, 18, year, old, bourbons., a, ma...",marriage 13 18 year old bourbons. a mature el...,"[ , marriage, 13, 18, year, old, bourbon, a, m...",marriage 13 18 year old bourbon a mature el...
1,2,Dave Broom,There have been some legendary Bowmores from t...,13500.0,97,42.90,1.0,"[legendary, bowmores, mid-60s, bit, equal., al...",legendary bowmores mid-60s bit equal. all sha...,"[ , legendary, bowmore, mid-60, bit, equal, al...",legendary bowmore mid-60 bit equal all shar...
2,3,John Hansell,This bottling celebrates master distiller Park...,150.0,97,50.00,2.0,"[bottling, celebrates, master, distiller, park...",bottling celebrates master distiller parker b...,"[ , bottling, celebrate, master, distiller, pa...",bottling celebrate master distiller parker ...
3,4,John Hansell,What impresses me most is how this whisky evol...,4500.0,97,40.50,1.0,"[impresses, whisky, evolves;, incredibly, comp...",impresses whisky evolves; incredibly complex....,"[ , impress, whisky, evolve, incredibly, compl...",impress whisky evolve incredibly complex on...
5,9,Fred Minnick,"A caramel-laden fruit bouquet, followed by une...",150.0,96,54.49,2.0,"[caramel-laden, fruit, bouquet,, followed, une...","caramel-laden fruit bouquet, followed unendin...","[ , caramel, laden, fruit, bouquet, follow, un...",caramel laden fruit bouquet follow unending...


In [171]:
target = 'string_category'
features = ['lammatized_sentence', 'price', 'ratingValue', 'pert_alcohol']

In [172]:
train_target = pd.DataFrame(train[target])

In [55]:
train_target.shape

(2586, 1)

In [315]:
train.isnull().sum()

id                     0
author                 0
description            0
price                  0
ratingValue            0
pert_alcohol           0
category               0
tokens                 0
tokenized_sentences    0
lemmas                 0
lammatized_sentence    0
string_category        0
dtype: int64

In [317]:
train[target].shape

(2586,)

In [161]:
# Import Statements
from sklearn.pipeline import Pipeline

from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [118]:
# class GridSearchCV(GridSearchCV):

#   def fit(self, *args, **kwargs):
#     return super().fit(*args, eval_set=eval_set, eval_metric='rmse', 
#           early_stopping_rounds=50, **kwargs)

In [ ]:

# model = XGBRegressor(n_estimators=5000, n_jobs=-1, max_depth=7)
# model.fit(X_train_encoded, y_train['outcome'], 
#           eval_set=eval_set, eval_metric='auc', early_stopping_rounds=100)

In [127]:
# X_train, X_test, y_train, y_test = train_test_split(train, train_target, train_size=0.8)

In [133]:
X_train.shape, y_train.shape

((2068, 11), (2068, 1))

In [128]:
# eval_set = [(X_test, y_train), 
#             (X_test, y_test)]

In [ ]:
# class XGRegressorEval(XGBRegressor):
#   def fit(self, *args, **kwargs):
#     return super().fit(*args, eval_set=eval_set, eval_metric='rmse', 
#           early_stopping_rounds=10, **kwargs)
    
# pipeline = make_pipeline(ce.OrdinalEncoder(), XGRegressorEval(n_estimators=1000, n_jobs=-1))

In [381]:

# Create Pipeline

vect = TfidfVectorizer(stop_words = 'english')
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

In [382]:
pipe.fit(train['lammatized_sentence'], train_target)

C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [383]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (100, 500, 1000),
    'clf__max_iter':(20, 10, 1000)
}
grid_search = GridSearchCV(pipe,parameters, cv=7, n_jobs=-1, verbose=1)
grid_search.fit(train['lammatized_sentence'], train_target)


Fitting 7 folds for each of 27 candidates, totalling 189 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed:   17.0s finished
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=7, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (100, 500, 1000), 'clf__max_iter': (20, 10, 1000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [202]:
# Instantiate
from sklearn.neighbors import KNeighborsClassifier

vect = TfidfVectorizer(stop_words = 'english')
# Fit on TF-IDF Vectors
nn  = KNeighborsClassifier( n_neighbors=5, algorithm='ball_tree')
pipe_nn = Pipeline([('vect', vect), ('clf', nn)])
pipe_nn.fit(train['lammatized_sentence'], train_target )

parameters = {
    'vect__max_df': (0.2, 0.75, 1.0),
    'vect__max_features': (100, 500, 1000),
    'clf__leaf_size':(30, 50, 100)
}
grid_search = GridSearchCV(pipe_nn, parameters, cv=7, n_jobs=-1, verbose=1, scoring='accuracy')
grid_search.fit(train['lammatized_sentence'], train_target)

C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\neighbors\base.py:217: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


Fitting 7 folds for each of 27 candidates, totalling 189 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed:   21.3s finished
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\pipeline.py:267: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\neighbors\base.py:217: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


GridSearchCV(cv=7, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...ki',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (100, 500, 1000), 'clf__leaf_size': (30, 50, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [384]:
category_mapper = {'Scotch': 1,
'Bourbon/Tennessee': 2,
'Craft Whiskey': 3,
'Canadian': 4}
y_pred = [category_mapper[letter] for letter in grid_search.predict(testing['lammatized_sentence'])]

In [ ]:
grid_search.score(pgrid_search.predict(testing['lammatized_sentence']))

In [187]:
category_mapper

{'Scotch': 1, 'Bourbon/Tennessee': 2, 'Craft Whiskey': 3, 'Canadian': 4}

In [189]:
y_pred = [category_mapper[letter] for letter in grid_search.predict(testing['lammatized_sentence'])]

In [385]:

submission3 = pd.DataFrame({'id':testing['id'],'category':y_pred})
submission3['category'] = submission3['category'].astype(int)
filename = 'kaggle submission 3.csv'

submission.to_csv(filename,index=False)

In [59]:
pipe.fit(train['lammatized_sentence'], train_target)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=0.05,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [77]:
test = pd.read_csv('test.csv')

In [78]:
test.shape

(288, 6)

In [74]:
y_pred = pipe.predict(test['description'])

In [80]:
submission = pd.DataFrame({'id':test['id'],'category':y_pred})

In [85]:
submission['category'] = submission['category'].astype(int)

In [86]:
filename = 'kaggle submission 1.csv'

submission.to_csv(filename,index=False)

In [87]:
submission['category'].dtype

dtype('int32')

In [88]:
pipe.fit(train['description'], train_target)

C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=0.05,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tru...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [89]:
y_pred = pipe.predict(test['description'])
submission2 = pd.DataFrame({'id':test['id'],'category':y_pred})
submission2['category'] = submission2['category'].astype(int)
filename = 'kaggle submission 2.csv'

submission.to_csv(filename,index=False)

In [375]:
pd.DataFrame(sparse, columns=sparse_columns).index, pd.DataFrame(sparce_author, columns=author_colums).index, train.index, sparce_author.shape, train_target.shape

(RangeIndex(start=0, stop=2586, step=1),
 RangeIndex(start=0, stop=2586, step=1),
 Int64Index([   0,    1,    2,    3,    5,    6,    7,    8,   10,   11,
             ...
             2864, 2865, 2866, 2867, 2868, 2869, 2870, 2871, 2872, 2873],
            dtype='int64', length=2586),
 (2586, 25),
 (2586, 1))

In [366]:
pd.concat([pd.DataFrame(sparse, columns=sparse_columns), pd.DataFrame(sparce_author, columns=author_colums)], axis=1).shape

(2586, 7330)

In [367]:
train.shape

(2586, 12)

In [470]:
vect = TfidfVectorizer(stop_words = 'english')
sparse = vect.fit_transform(train['lammatized_sentence']).todense()
sparse_columns= vect.get_feature_names()
vect = TfidfVectorizer(stop_words = 'english')
sparce_author = vect.fit_transform(train['lammatized_author']).todense()
author_colums = vect.get_feature_names()
dtm = pd.concat([pd.DataFrame(sparse, columns=sparse_columns), pd.DataFrame(sparce_author, columns=author_colums)], axis=1)
dtm2 = pd.concat([dtm, train], axis=1)

In [471]:
dtm2.columns

Index(['00', '000', '005', '011', '035', '070', '076', '08', '080', '09',
       ...
       'price', 'ratingValue', 'pert_alcohol', 'category', 'tokens',
       'tokenized_sentences', 'lemmas', 'lammatized_sentence',
       'string_category', 'lammatized_author'],
      dtype='object', length=7343)

In [483]:
train.shape, dtm2.shape

((2586, 13), (2586, 7343))

In [473]:
features = author_colums + sparse_columns

In [474]:
features.extend(['price', 'ratingValue', 'pert_alcohol'])

In [475]:
features[-1]

'pert_alcohol'

In [476]:
vect = TfidfVectorizer(stop_words = 'english')
sparce_author = vect.fit_transform(testing['lammatized_author']).todense()
author_colums = vect.get_feature_names()
vect = TfidfVectorizer(stop_words = 'english')
sparce = vect.fit_transform(testing['lammatized_sentence']).todense()
sparce_colums = vect.get_feature_names()
testing2 = pd.concat([pd.DataFrame(sparse, columns=sparse_columns), pd.DataFrame(sparce_author, columns=author_colums)], axis=1)
testing3 = pd.concat([testing2, testing], axis=1)

In [488]:
dtm2.index

RangeIndex(start=0, stop=2586, step=1)

In [489]:
testing3.index

RangeIndex(start=0, stop=2586, step=1)

In [478]:
sgdc = SGDClassifier(n_jobs=-1)

In [479]:
dtm2.isnull().sum().head()

00     0
000    0
005    0
011    0
035    0
dtype: int64

In [480]:
sgdc.fit(dtm2[features], train_target)

C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
C:\Users\mhdal\Anaconda3\envs\lambda-nlp\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=-1, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [501]:
features.pop('polonski', 'skiver', 'susannah')

TypeError: pop() takes at most 1 argument (3 given)

In [505]:
y_pred = [category_mapper[letter] for letter in sgdc.predict(testing3[[x for x in features if x not in ['polonski', 'skiver', 'susannah']]])]

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [448]:
len(testing['id'])

288

In [443]:
submission4 = pd.DataFrame({'id':test['id'],'category':y_pred})
submission4['category'] = submission2['category'].astype(int)
filename = 'kaggle submission 4.csv'

submission.to_csv(filename,index=False)

ValueError: array length 2586 does not match index length 288

In [420]:
parameters = {
    'penalty': ['l2']
}
grid_search = GridSearchCV(sgdc, parameters, cv=7, n_jobs=-1, verbose=1)
grid_search.fit(dtm2[features], train_target)

Fitting 7 folds for each of 1 candidates, totalling 7 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


MemoryError: 